In [ ]:
import ROOT
from draw_funcs import draw_chi2_gaussian_sim, draw_1d_hist, draw_2d_hist2
from ROOT import TFile
import matplotlib.pyplot as plt

In [ ]:
entry = 6
nevents = 1000000
mfield = 1.3
input_path = '/var/kinfit/data/tr_ph_run021142.root'
output_path = 'kinfit_gaussian_fit_etapipi_etato2gamma_run021142_.root'

In [ ]:
ROOT.gInterpreter.ProcessLine('gROOT->LoadMacro("/home/hep/packages/gaussgen/share/gaussgen/rootlogon.C")')

In [ ]:
ROOT.gInterpreter.ProcessLine('gROOT->LoadMacro("/home/hep/packages/kfcmd/share/kfcmd/rootlogon.C")')

In [ ]:
ROOT.gInterpreter.ProcessLine('.L TrPh.C++')

In [ ]:
ROOT.gInterpreter.ProcessLine('TFile fl("{}", "read")'.format(input_path))

In [ ]:
ROOT.gInterpreter.ProcessLine('TrPh a(tr_ph)')

In [ ]:
ROOT.gInterpreter.ProcessLine('a.setNEvents({})'.format(nevents))

In [ ]:
ROOT.gInterpreter.ProcessLine('a.setEntry({})'.format(entry))

In [ ]:
ROOT.gInterpreter.ProcessLine('a.Loop("{output_path}", {mfield})'.format(output_path=output_path, mfield=mfield))

In [ ]:
ROOT.gInterpreter.ProcessLine('fl.Close()')

In [ ]:
fl = TFile.Open(output_path, 'read')

In [ ]:
ROOT.gInterpreter.ProcessLine('''double fcn(double *x, double *p) {
                                        return p[0] / 0.1875 * ROOT::Math::chisquared_pdf(x[0], p[1]);}''')

In [ ]:
ROOT.gInterpreter.ProcessLine('TF1 f0("f0", fcn, 0, 24, 2)')

In [ ]:
ROOT.gInterpreter.ProcessLine('f0.SetNpx(10000)')
ROOT.gInterpreter.ProcessLine('f0.SetParameter(0, 2.e+4)')
ROOT.gInterpreter.ProcessLine('f0.SetParameter(1, 6)')

In [ ]:
ROOT.gInterpreter.ProcessLine('kf_chi2->Fit(&f0)')

In [ ]:
draw_chi2_gaussian_sim("kf_chi2", "f0", xlabel=r'$\chi^2$',
                       fraction=0.8,
                       info_coords=(0.5, 0.2),
                       dw=-0.2,
                      dh=0.4)
plt.savefig('kf_chi2_gsim_etapipi.eps')

In [ ]:
draw_1d_hist("qhist", 
             xlabel=r'$\frac{1}{2}\Delta\bm{y}^{\intercal}\hat{Q}\Delta\bm{y}$',
            fraction=0.47,
            info_coords=(0.4, 0.4))
plt.gcf().subplots_adjust(bottom=0.275)
plt.savefig("qhalf_gsim_etapipi.eps")

In [ ]:
import matplotlib.colors as colors
import numpy as np
bounds = np.linspace(0, 8, 16)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
draw_2d_hist2('chi2_vs_q', 
              xlabel=r'$\chi^2$', 
              ylabel=r'$\frac{1}{2}\Delta\bm{y}^{\intercal}\hat{Q}\Delta\bm{y}$', 
              fraction=0.47,
              bins=(256, 256),
              norm=norm,
              clip=((0, 30.), (-5., 5.)))
plt.gcf().subplots_adjust(left=0.235)
plt.savefig('qhalf_vs_chi2_gsim_etapipi.eps')

In [ ]:
# fl.Close()